In [1]:
import openai
from qdrant_client import QdrantClient

# Embedding function

In [2]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(input=text, model=model)
    return response.data[0].embedding

# Retrieval function

In [7]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [17]:
def retrieve_data(query, qdrant_client, k=5):
    query_embeddings = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00", query=query_embeddings, limit=k
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }

In [19]:
retrieved_context = retrieve_data(
    "What kind of earphones can I get?", qdrant_client, k=10
)

retrieved_context

{'retrieved_context_ids': ['B0C52WNWY6',
  'B09Y57K53R',
  'B0B51PZC3L',
  'B0BXRMTZCD',
  'B0B3J1NNGL',
  'B0C3GP9QFB',
  'B0C999S8YM',
  'B0B93MRTWF',
  'B0BM961XHH',
  'B09ZNKRFSC'],
 'retrieved_context': ["Wireless Earbud, Bluetooth Headphones in Ear Noise Cancelling Mic Mini Ear Buds, Bluetooth 5.1 Light Weight Deep Bass, IP7 Waterproof Wireless Headphones, 30H Playtime for Workout/Home/Office, WhiteCVC8.0 Noise Reduction and Hi-Fi Stereo Sound: Adopt advanced Bluetooth chip and high-sensitivity diaphragm and filter out outside noise allowing the other side to hear you more clearly. Besides, the built-in high-quality microphone can provide you with authentic sound and powerful bass. Hear your original voice whether you video with friends or make phone calls. Great Battery Life and Dual LED Power Display: The wireless earbuds can last 6-7 hours of high quality sound output on a single charge. Features with smart LED digital display can help check the power of charging case at any t

# Format retrieved context function


In [20]:
def process_context(context):
    formatted_context = ""
    for id, chunk in zip(
        context["retrieved_context_ids"], context["retrieved_context"]
    ):
        formatted_context += f"- {id}: {chunk} \n"

    return formatted_context

In [21]:
preprocessed_context = process_context(retrieved_context)

In [22]:
print(process_context(retrieved_context))

- B0C52WNWY6: Wireless Earbud, Bluetooth Headphones in Ear Noise Cancelling Mic Mini Ear Buds, Bluetooth 5.1 Light Weight Deep Bass, IP7 Waterproof Wireless Headphones, 30H Playtime for Workout/Home/Office, WhiteCVC8.0 Noise Reduction and Hi-Fi Stereo Sound: Adopt advanced Bluetooth chip and high-sensitivity diaphragm and filter out outside noise allowing the other side to hear you more clearly. Besides, the built-in high-quality microphone can provide you with authentic sound and powerful bass. Hear your original voice whether you video with friends or make phone calls. Great Battery Life and Dual LED Power Display: The wireless earbuds can last 6-7 hours of high quality sound output on a single charge. Features with smart LED digital display can help check the power of charging case at any time, so you always know when it's time to recharge.Take it with you, you can relax enjoy music while traveling. IP7 Waterproof Earbuds and Ergonomic Design: Bluetooth earbuds boast IP7 rated techn

# Create Prompt function


In [23]:
def build_prompt(preprocessed_context, question):
    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [24]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [25]:
print(build_prompt(preprocessed_context, "What kind of earphones can I get?"))


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- B0C52WNWY6: Wireless Earbud, Bluetooth Headphones in Ear Noise Cancelling Mic Mini Ear Buds, Bluetooth 5.1 Light Weight Deep Bass, IP7 Waterproof Wireless Headphones, 30H Playtime for Workout/Home/Office, WhiteCVC8.0 Noise Reduction and Hi-Fi Stereo Sound: Adopt advanced Bluetooth chip and high-sensitivity diaphragm and filter out outside noise allowing the other side to hear you more clearly. Besides, the built-in high-quality microphone can provide you with authentic sound and powerful bass. Hear your original voice whether you video with friends or make phone calls. Great Battery Life and Dual LED Power Display: The wireless earbuds can last 6-7 hours of high quality sound output on a

# Generate answer function (from LLM)

In [28]:
def generate_answer(prompt):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )
    return response.choices[0].message.content

In [30]:
print(generate_answer(prompt))

You can choose from several types of earphones available:

1. Wireless Earbuds with Noise Cancelling and Deep Bass:
   - Bluetooth 5.1 technology, IP7 waterproof, 30 hours playtime.
   - Features CVC 8.0 noise reduction, Hi-Fi stereo sound, touch controls.
   - Suitable for workout, home, office.
   - Example: Wireless Earbud B0C52WNWY6.

2. Bone Conduction Ear Hook Headphones:
   - Non in-ear plug design, lightweight and painless to wear.
   - Bluetooth 5.0, waterproof, 10 hours continuous playback.
   - Great for sports and workouts.
   - Example: XpacEbank Wireless Bluetooth Headphone B09Y57K53R.

3. Wireless Earbuds with Ear Hooks and Advanced Bluetooth 5.3:
   - IPX7 waterproof, ergonomic earhook design for secure fit.
   - 120 hours total playtime with charging case, touch control.
   - Powerful bass and clear calls with noise cancellation.
   - Example: VOESUD 2022 Upgrade Wireless Earbuds B0B51PZC3L.

4. Sleep Headband Headphones:
   - Adjustable headband with ultra-thin speake

# Combined RAG Pipeline

In [32]:
def rag_pipeline(question, top_k=5):
    """
    Complete RAG (Retrieval-Augmented Generation) pipeline for answering questions about products.

    This function implements a full RAG pipeline that:
    1. Retrieves relevant product information from a Qdrant vector database
    2. Processes the retrieved context into a formatted string
    3. Builds a prompt with the context and question
    4. Generates an answer using OpenAI's GPT model

    Args:
        question (str): The user's question about products
        top_k (int, optional): Number of most similar products to retrieve. Defaults to 5.

    Returns:
        str: Generated answer based on retrieved product information

    Example:
        >>> answer = rag_pipeline("What kind of earphones can I get?", top_k=10)
        >>> print(answer)
    """
    qdrant_client = QdrantClient(url="http://localhost:6333")
    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    processed_context = process_context(retrieved_context)
    prompt = build_prompt(processed_context, question)
    answer = generate_answer(prompt)

    return answer

In [33]:
print(rag_pipeline("Can I get a tablet"))

Yes, you can get a tablet. There are two tablets available:

1. DUODUOGO 2 in 1 Tablet 10 Inch with Keyboard Mouse Stylus, 128GB Expand 64GB ROM 4GB RAM Android 10.0 Quad-Core HD IPS Screen, 8MP Dual Camera, GPS, FM, OTG, Bluetooth, 4G Dual SIM & WiFi (Model: DGO-P610).

2. DOOGEE T10 Android Tablet 2023, 10.1" FHD+ Android 12 Tablet with 15GB RAM, 128GB storage, Octa-Core processor, 8300mAh Battery, 2.4G/5G WiFi, Bluetooth, GPS, Split Screen, and TÜV Low Blue Light certification.

You can choose either based on your preferences for features and specifications.
